In [1]:
import numpy as np
import pickle
dict_name = 'model_data_dictionary_N_Pa.pkl'
data_dict = pickle.load(open(dict_name, 'rb'))
sim_membranes = pickle.load(open('sim_membranes.pkl', 'rb'))
new_dict = data_dict
keys=list(new_dict.keys())
N_to_lbf = 0.224809
lbf_to_N = 1/N_to_lbf
PSI_to_Pa = 6894.76
Pa_to_PSI = 1/PSI_to_Pa
import numpy as np

# Create flattened pressure and force variables
locals().update({f"pressure{i+1}": np.array(new_dict[key][0][1]).flatten() for i, key in enumerate(keys)})
locals().update({f"force{i+1}": np.array(new_dict[key][1]).flatten() for i, key in enumerate(keys)})
locals().update({f"height{i+1}": [x[0] for x in new_dict[key][0][0]] for i, key in enumerate(keys)})

# Then create plane tuples
for i in range(len(keys)):
    locals()[f"plane{i+1}"] = (
        keys[i],
        (locals()[f"pressure{i+1}"], locals()[f"force{i+1}"], locals()[f"height{i+1}"])
    )

# keep planes for which simulation solution was possible
exp_data=[plane1,plane2,plane3,plane4,plane5,plane6,plane7,
plane8,plane9,plane10,plane11,plane14,plane15,plane17,plane18,
plane19,plane20,plane22]


In [2]:
# Convert experimental data to match simulation data structure
standardized_exp_data = []
for name, data in exp_data:
    # Convert each component (pressure, force, heights) to numpy array
    standardized_data = (
        np.array(data[0]),  # pressure
        np.array(data[1]),  # force
        np.array(data[2])   # heights
    )
    standardized_exp_data.append((name, np.array(standardized_data)))

standardized_sim_data = []
for name, data in sim_membranes:
    # Convert each component (pressure, force, heights) to numpy array
    standardized_sim = (
        np.array(data[0]),  # force
        np.array(data[1]),  # pressure
        np.array(data[2])   # heights
    )
    standardized_sim_data.append((name, np.array(standardized_sim)))

# Both sim_data and standardized_exp_data have shape (key, (pressure, force, heights)) where key is a string that shows
# thickness and the location of the rings.

In [3]:
def calculate_normalized_rmse(plane, sim_plane):
    min_errors = []
    force_differences = []
    
    # Normalize the planes
    norm_plane = (
        plane[1][0]/np.max(plane[1][0]),
        plane[1][1],
        plane[1][2]/(1000*np.max(plane[1][2]))
    )
    
    norm_sim_plane = (
        sim_plane[1][1]/np.max(plane[1][0]),
        sim_plane[1][0],
        sim_plane[1][2]/np.max(plane[1][2])
    )
    
    for i in range(len(norm_plane[0])):
        # Finding closest point in the P vs height plane
        errors = np.sqrt((norm_plane[0][i] - np.array(norm_sim_plane[0]))**2 + 
                        (norm_plane[2][i] - np.array(norm_sim_plane[2]))**2)
        
        j_min = np.argmin(errors)
        min_errors.append(errors[j_min])
        

        # GMC 3/24 - add a tolerance to ensure that the closest point is actually close
        if errors[j_min] < 0.1:
            # Calculate force difference using normalized values
            force_diff = abs(norm_plane[1][i] - norm_sim_plane[1][j_min])
            force_differences.append(force_diff)

        # Calculate force difference using normalized values
        force_diff = abs(norm_plane[1][i] - norm_sim_plane[1][j_min])
        force_differences.append(force_diff)
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean(np.array(force_differences)**2))
    return rmse, np.array(min_errors), np.array(force_differences)


In [4]:
rmse_results = [[],[],[]]
for i in range(len(standardized_sim_data)):
    rmse,errors,force_diffs = calculate_normalized_rmse(standardized_exp_data[i], standardized_sim_data[i])
    rmse_results[0].append(rmse)
    rmse_results[1].append(errors)
    rmse_results[2].append(force_diffs)

In [5]:
print(f"Six ringless membranes RMSEs: ({rmse_results[0][0]:.2f}N, {rmse_results[0][1]:.2f}N, {rmse_results[0][2]:.2f}N, {rmse_results[0][3]:.2f}N, {rmse_results[0][4]:.2f}N, {rmse_results[0][5]:.2f}N)")
print(f"Twelve ringed membranes RMSEs: ({rmse_results[0][6]:.2f}N, {rmse_results[0][7]:.2f}N, {rmse_results[0][8]:.2f}N, {rmse_results[0][9]:.2f}N, {rmse_results[0][10]:.2f}N, {rmse_results[0][11]:.2f}N, {rmse_results[0][12]:.2f}N, {rmse_results[0][13]:.2f}N, {rmse_results[0][14]:.2f}N, {rmse_results[0][15]:.2f}N, {rmse_results[0][16]:.2f}N, {rmse_results[0][17]:.2f}N)")
print(f"Ringless membranes: Average RMSE: {np.mean(rmse_results[0][:6]):.2f}N")
print(f"Ringed membranes: Average RMSE: {np.mean(rmse_results[0][6:18]):.2f}N")
print(f"Total average RMSE: {np.mean(rmse_results[0]):.2f}N")

Six ringless membranes RMSEs: (1.71N, 3.11N, 5.17N, 5.94N, 5.12N, 14.13N)
Twelve ringed membranes RMSEs: (4.36N, 8.30N, 6.35N, 1.23N, 1.93N, 4.58N, 1.44N, 4.74N, 1.94N, 1.96N, 4.74N, 1.64N)
Ringless membranes: Average RMSE: 5.86N
Ringed membranes: Average RMSE: 3.60N
Total average RMSE: 4.35N
